In [1]:
import pandas as pd

In [2]:
file_stockActual = '08_11/Inventario_0811.xlsx'
inventario_actual = pd.read_excel(file_stockActual)
inventario_actual = inventario_actual[['Código', 'Producto', 'Existencia', 'P. Costo', 'P. Venta']]

inventario_actual.columns = ['code', 'product', 'qty', 'cost', 'price']

print(inventario_actual)


            code                              product qty     cost    price
0      175002710  Pisco Mistral Nobel Barrica Tostada   7   $9.622  $13.000
1        1801828                           Budwuaisen  17     $890   $1.000
2     2490238841                       Red Bull 473ml   9   $1.650   $2.200
3       30052765                    Ocb 1 1/4 Premium  21     $439   $1.000
4       30068926                            Ocb Xpert  14     $412   $1.000
..           ...                                  ...  ..      ...      ...
890     90446849          >Red Bull Red Edition 250ml   3   $1.240   $1.800
891     90453823            Red Bull Sugar Free 473ml  10   $1.997   $2.600
892     90454707             Redbull Sabor Fruta250ml  20   $1.126   $1.800
893     90456695                Red Bull Winter 250ml  24   $1.240   $1.800
894  96749021345                Whiskey Evan Williams   3  $13.000  $18.000

[895 rows x 5 columns]


In [3]:
#Archivos de inventario Capturados

file1 = '08_11/Manuel_01.xls'
file2 = '08_11/Manuel_02.xls'
file3 = '08_11/Roberto.xls'
file4 = '08_11/Elena.xls'

inventario1 = pd.read_excel(file1)
inventario2 = pd.read_excel(file2)
inventario3 = pd.read_excel(file3)
inventario4 = pd.read_excel(file4)

TotalInventario = pd.concat([inventario1, inventario2, inventario3, inventario4])
TotalInventario = TotalInventario[['Código de Barras:', 'Artículo:', 'Unidades:']]

#se cambian los nombres de las columnas
TotalInventario.columns = ['code', 'product', 'qty']

#se convirtene los datos de la columna 'code' y  'qty' a numericos
TotalInventario['code'] = TotalInventario['code'].astype(str)
TotalInventario['qty'] = pd.to_numeric(TotalInventario['qty'], errors='coerce', downcast='integer')
print(TotalInventario)


              code                         product  qty
0    7501233712713  Tequila blanco el merendero 1L    2
1    5000289020701             Gordons dry gin 750    2
2    5000291026548           Tanque Ray dark berry    1
3    5000281015248                Tanqueray london    3
4    7640175740047                 Bomba vapor gin    2
..             ...                             ...  ...
180  7801620006631          cachantun más granada     4
181  7804646000256                   mister Big 2L    1
182  7804603921082                    Sandy Mac 1L    2
183  5010408252074                   Sandy Mac 750    2
184  6932757154818      pañuelo desechable care up   40

[572 rows x 3 columns]


In [4]:
# se comprueba si los codigos de totalinventario estan en inventario_actual

Faltantes = []
exclude = ['8501110080255','8804374087016','7804300155889', '4841676004925', '11624822', '9807181688642', '3827372569853', '5571147', '2722764', '7707111', '2228620006600', ]
for i in TotalInventario['code']:
     if i not in exclude:
          if i not in inventario_actual['code'].values:
               Faltantes.append(i)

print(len(Faltantes))
print(Faltantes)

#se agrupan los codigos unicos de totalinventario y se suman los 'qty' de cada uno de ellos
TotalInventario = TotalInventario.groupby('code').sum().reset_index()
#print(TotalInventario)

#se exporta totalinventario a un archivo excel
TotalInventario.to_excel('total_inventario.xlsx', index=False) 


0
[]


In [5]:
#se abre el archivo original de inventario 
form_file = pd.read_excel(file_stockActual)

#se establece la columna 'existencia' a 0 para todos los codigos
form_file['Existencia'] = 0

# se remplaza la columna 'existencia' con los valores de 'qty' de totalinventario
for i in range(len(TotalInventario)):
     form_file.loc[form_file['Código'] == TotalInventario['code'][i], 'Existencia'] = TotalInventario['qty'][i]


#se elim,ina $ y . de la columna 'P. Costo' y 'P. Venta' y P.Mayoreo y se convierten a numericos
form_file['P. Costo'] = form_file['P. Costo'].str.replace('$', '').str.replace('.', '').astype(int)
form_file['P. Venta'] = form_file['P. Venta'].str.replace('$', '').str.replace('.', '').astype(int)
form_file['P. Mayoreo'] = form_file['P. Mayoreo'].str.replace('$', '').str.replace('.', '').astype(int)

# se establece a numerico la columna Inv. Mínimo y Inv. Máximo
form_file['Inv. Mínimo'] = pd.to_numeric(form_file['Inv. Mínimo'], errors='coerce', downcast='integer')
form_file['Inv. Máximo'] = pd.to_numeric(form_file['Inv. Máximo'], errors='coerce', downcast='integer')


#se exporta el archivo con el inventario actualizado
form_file.to_excel('Inventario_0811_actualizado.xlsx', index=False, engine='openpyxl')